In [1]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
from tma.object import Object
from tma.functions import get_df
from tma.model import Model
from tma.algorithms import swarm
import time
from pathlib import Path
from datetime import datetime

In [2]:
print("Время последнего моделирования:", datetime.now().strftime("%d/%m/%Y %H:%M"))

# Создаем наблюдателя
observer_x, observer_y, observer_course, observer_velocity = 0, 0, 0, 5
observer = Object('Наблюдатель', observer_x, observer_y, observer_course,
                observer_velocity, verbose=True)

# Моделирование траектории наблюдателя
observer.forward_movement(3 * 60)
observer.change_course(270, 'left', omega=0.5)
observer.forward_movement(5 * 60)
observer.change_course(90, 'right', omega=0.5)
observer.forward_movement(3 * 60)

Время последнего моделирования: 24/04/2021 16:22
Наблюдатель движется прямо по курсу 0.0° 180с
Наблюдатель перешёл на курс 270.0° за 180с
Наблюдатель движется прямо по курсу 270.0° 300с
Наблюдатель перешёл на курс 90.0° за 360с
Наблюдатель движется прямо по курсу 90.0° 180с


In [3]:
p0 = [0., 25., 90., 7.]
d_arr = [10., 20., 30., 40.]
std_arr = [0., 0.1, 0.2, 0.3, 0.5, 1.]
alg_arr = ['ММП']
def target_func(seed=None):
            np.random.seed(seed)
            b = 0
            c = np.random.uniform(0, 180)
            v = np.random.uniform(3, 15)
            return [b, target_func.d, c, v]
result_df = pd.DataFrame(columns=['Algorithm', 'std', 'd', 'sr_0', 'sr_1', 'sr_2', 'sr_3', 'Ка', 'Nf', 'Nf_max', 'Iter', 'Iter_max', 't', 't_max'])
mean_columns = ['Ка', 'Nf', 'Iter', 't']
max_columns = ['Nf', 'Iter', 't']

In [5]:
model = Model(observer)
for d in d_arr:
    target_func.d = d
    for std in std_arr:
        model.standart_deviation = np.radians(std)
        for alg in alg_arr:
            dict_results = swarm(model, algorithm_name=alg, n=100, target_func=target_func, p0=p0, seeded=True)
            df = get_df(dict_results)
            res = pd.DataFrame({'Algorithm': [alg], 'std': [std], 'd': [d]})
            for i in range(4):
                res['sr_' + str(i)] = [df['Успех'].apply(lambda x: x[i]).mean(axis=0)]
            res[mean_columns] = df[mean_columns].mean(axis=0)
            res[['Nf_max', 'Iter_max', 't_max']] = df[max_columns].apply(max, axis=0)
            result_df = result_df.append(res)

KeyError: 'ММП в реальном времени'

In [9]:
result_df['Ка'] = result_df['Ка'].apply(np.sqrt)
result_df.rename(columns={'Ка': 'sr_Ka'}, inplace=True)
result_df.reset_index(drop=True)
Path("../tests/test6").mkdir(parents=True, exist_ok=True)
result_df.to_excel('../tests/test6/results.xlsx', index=False)

In [27]:
display_df = result_df.copy()
precisions = {'sr_Ka' : '{0:.2f}', 'Nf' : '{0:.0f}', 'Nf_max': '{0:.0f}',\
'Iter' : '{0:.0f}', 'Iter_max' : '{0:.0f}', 't' : '{0:.4f}', 't_max' : '{0:.4f}'}
for col, precision in precisions.items():
    display_df[col] = display_df[col].apply(lambda x: precision.format(x))
display_df

,Algorithm,std,d,sr_0,sr_1,sr_2,sr_3,sr_Ka,Nf,Nf_max,Iter,Iter_max,t,t_max
0,ММП v2,0.0,10,1.000,1.000,1.000,1.000,0.04,15,28,12,21,0.0029,0.0060
1,Метод N пеленгов,0.0,10,1.000,1.000,1.000,1.000,0.00,nan,nan,nan,nan,0.0002,0.0006
2,ММП v2,0.1,10,1.000,1.000,1.000,1.000,0.56,15,28,12,21,0.0029,0.0057
3,Метод N пеленгов,0.1,10,0.967,0.997,0.997,1.000,0.57,nan,nan,nan,nan,0.0002,0.0005
4,ММП v2,0.2,10,1.000,1.000,1.000,1.000,0.67,15,28,12,21,0.0029,0.0058
5,Метод N пеленгов,0.2,10,0.757,0.966,0.970,0.994,0.68,nan,nan,nan,nan,0.0002,0.0006
6,ММП v2,0.3,10,0.996,1.000,1.000,1.000,0.74,15,28,12,21,0.0029,0.0058
7,Метод N пеленгов,0.3,10,0.521,0.834,0.839,0.960,0.75,nan,nan,nan,nan,0.0002,0.0005
8,ММП v2,0.5,10,0.933,0.997,0.997,1.000,0.84,15,28,12,21,0.0029,0.0056
9,Метод N пеленгов,0.5,10,0.095,0.412,0.420,0.643,0.86,nan,nan,nan,nan,0.0002,0.0005
